<a href="https://colab.research.google.com/github/CaloCare/MachineLearning/blob/main/Nutrition_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [5]:
!pip install keras-tuner --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch

<ipython-input-6-84c4085b67f7>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


##LOAD DATASET

In [9]:
!pip install kaggle

In [13]:
import os
os.makedirs('/root/.kaggle/', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [14]:
!kaggle datasets download -d anasfikrihanif/indonesian-food-and-drink-nutrition-dataset


Dataset URL: https://www.kaggle.com/datasets/anasfikrihanif/indonesian-food-and-drink-nutrition-dataset
License(s): CC0-1.0
100% 73.7k/73.7k [00:00<00:00, 490kB/s]
100% 73.7k/73.7k [00:00<00:00, 489kB/s]


In [16]:
import zipfile

with zipfile.ZipFile('indonesian-food-and-drink-nutrition-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('indonesian-food-and-drink-nutrition-dataset')

# Muat dataset ke dalam DataFrame
data = pd.read_csv('/content/indonesian-food-and-drink-nutrition-dataset/nutrition.csv')


##DISTRIBUSI DATA

In [24]:
# Hitung total nutrisi
data['total_nutrition'] = data['calories'] + data['proteins'] + data['fat'] + data['carbohydrate']

# Buat kategori dari total nutrisi
bins = [0, 200, 400, 600, 800, np.inf]
labels = [1, 2, 3, 4, 5]
data['evaluation'] = pd.cut(data['total_nutrition'], bins=bins, labels=labels)

# Konversi kolom 'evaluation' ke tipe data numerik
data['evaluation'] = data['evaluation'].cat.codes

# Misalkan target kolom yang ingin diprediksi adalah 'evaluation'
X = data[['calories', 'proteins', 'fat', 'carbohydrate']]
y = data['evaluation']

# Bagi dataset menjadi train dan validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


##HYPER PARAMETER TUNING

In [25]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(5, activation='softmax'))  # Output layer for 5 classes
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model



In [26]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Jumlah percobaan
    executions_per_trial=3,  # Eksekusi per percobaan
    directory='my_dir',
    project_name='hyperparam_tuning'
)

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

tuner.results_summary()


Trial 10 Complete [00h 00m 33s]
val_accuracy: 0.8888888955116272

Best val_accuracy So Far: 0.8925925890604655
Total elapsed time: 00h 06m 21s
Results summary
Results in my_dir/hyperparam_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 06 summary
Hyperparameters:
units: 384
Score: 0.8925925890604655

Trial 08 summary
Hyperparameters:
units: 288
Score: 0.8913580377896627

Trial 09 summary
Hyperparameters:
units: 224
Score: 0.8888888955116272

Trial 04 summary
Hyperparameters:
units: 416
Score: 0.8876543243726095

Trial 03 summary
Hyperparameters:
units: 320
Score: 0.8864197532335917

Trial 05 summary
Hyperparameters:
units: 160
Score: 0.8703703880310059

Trial 02 summary
Hyperparameters:
units: 96
Score: 0.8592592676480612

Trial 07 summary
Hyperparameters:
units: 64
Score: 0.8567901253700256

Trial 01 summary
Hyperparameters:
units: 256
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tun

In [27]:
# Dapatkan model terbaik
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluasi model terbaik pada validation set
val_loss, val_acc = best_model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_acc}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9231 - loss: 0.3096  
Validation Loss: 0.3544178009033203
Validation Accuracy: 0.9037036895751953


##TESTING

In [28]:
def predict_evaluation(calories, proteins, fat, carbohydrate):
    # Bentuk input pengguna menjadi array yang dapat diterima oleh model
    input_data = np.array([[calories, proteins, fat, carbohydrate]])

    # Prediksi menggunakan model yang telah dilatih
    prediction = best_model.predict(input_data)

    # Konversi prediksi ke kategori evaluasi
    predicted_category = np.argmax(prediction, axis=1)[0] + 1  # +1 karena kategori dimulai dari 1

    return predicted_category


In [29]:
def user_input():
    print("Masukkan nilai-nilai nutrisi untuk mendapatkan evaluasi:")

    # Meminta input dari pengguna
    calories = float(input("Kalori (cal): "))
    proteins = float(input("Protein (g): "))
    fat = float(input("Lemak (g): "))
    carbohydrate = float(input("Karbohidrat (g): "))

    # Prediksi evaluasi
    evaluation = predict_evaluation(calories, proteins, fat, carbohydrate)

    # Tampilkan hasil evaluasi
    print(f'Nilai evaluasi untuk nutrisi yang diberikan adalah: {evaluation}')

# Panggil fungsi user_input untuk menjalankan prediksi berbasis input pengguna
user_input()


Masukkan nilai-nilai nutrisi untuk mendapatkan evaluasi:
Kalori (cal): 175
Protein (g): 3
Lemak (g): 0.045
Karbohidrat (g): 40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Nilai evaluasi untuk nutrisi yang diberikan adalah: 2


In [30]:
def user_input():
    print("Masukkan nilai-nilai nutrisi untuk mendapatkan evaluasi:")

    # Meminta input dari pengguna
    calories = float(input("Kalori (cal): "))
    proteins = float(input("Protein (g): "))
    fat = float(input("Lemak (g): "))
    carbohydrate = float(input("Karbohidrat (g): "))

    # Prediksi evaluasi
    evaluation = predict_evaluation(calories, proteins, fat, carbohydrate)

    # Deskripsi hasil evaluasi
    descriptions = {
        1: "buruk untuk Anda",
        2: "tidak terlalu baik untuk Anda",
        3: "cukup baik untuk Anda",
        4: "baik untuk Anda",
        5: "sangat baik untuk Anda"
    }

    # Tampilkan hasil evaluasi dengan deskripsi
    print(f'Nilai evaluasi untuk nutrisi yang diberikan adalah: {evaluation} ({descriptions[evaluation]})')

# Panggil fungsi user_input untuk menjalankan prediksi berbasis input pengguna
user_input()


Masukkan nilai-nilai nutrisi untuk mendapatkan evaluasi:
Kalori (cal): 175
Protein (g): 3
Lemak (g): 0.045
Karbohidrat (g): 40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Nilai evaluasi untuk nutrisi yang diberikan adalah: 2 (tidak terlalu baik untuk Anda)
